In [5]:


"""
DOL Bids - Streamlit App (single-file)
Copy into `app.py` and run: `streamlit run app.py`

Features:
- Accept Copart URL or lot number (attempts fetch; falls back to manual input).
- Editable assumptions: FX, fees, shipping, reserve, target profit range.
- Repair estimator with low/base/high and editable line items.
- Copart fee calculator, trucking, shipping & clearing stubs.
- Profit back-solve for Max Bid; Avg Bid computed per spec.
- Sensitivity chart (FX swings) and CSV/Markdown export.
- Simple stubs for Copilot / web fetch (opt-in).
"""

import streamlit as st
from dataclasses import dataclass, asdict
from typing import List, Dict, Any, Optional
import math
import datetime
import pandas as pd
import io
import logging
import requests
import matplotlib.pyplot as plt

# -------------------------
# Logging
# -------------------------
logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")
logger = logging.getLogger("dol_bids_streamlit")

# -------------------------
# Data models
# -------------------------
@dataclass
class LotData:
    url: Optional[str] = None
    lot_id: Optional[str] = None
    year: Optional[int] = None
    make: Optional[str] = None
    model: Optional[str] = None
    trim: Optional[str] = None
    vin: Optional[str] = None
    mileage: Optional[int] = None
    primary_damage: Optional[str] = None
    secondary_damage: Optional[str] = None
    damage_codes: List[str] = None
    title: Optional[str] = None
    run_and_drive: Optional[bool] = None
    engine: Optional[str] = None
    drivetrain: Optional[str] = None
    sale_date: Optional[str] = None
    currency: str = "CAD"
    notes: Optional[str] = None

@dataclass
class RepairLineItem:
    part: str
    qty: int
    unit_cost: float  # lot currency
    labor_hours: float
    labor_rate: float  # per hour in lot currency
    def total_cost(self) -> float:
        return self.qty * self.unit_cost + self.labor_hours * self.labor_rate

@dataclass
class RepairEstimate:
    low: float
    base: float
    high: float
    items: List[RepairLineItem]
    contingency_pct: float

# -------------------------
# Default configuration (editable in UI)
# -------------------------
DEFAULT_CONFIG = {
    "currency": "CAD",
    "fx_rate_ngn_per_lot": 560.0,
    "fx_timestamp": datetime.datetime.utcnow().isoformat(),
    "target_profit_min_ngn": 8_000_000,
    "target_profit_max_ngn": 10_000_000,
    "resale_scenarios_ngn": [15_000_000, 25_000_000],
    "reserve_pct": 0.15,
    "copart_fee_table": [(0, 99.99, 100.0), (100, 499.99, 200.0), (500, 999.99, 300.0), (1000, 4999.99, 500.0), (5000, math.inf, 1000.0)],
    "internet_bid_fee": 60.0,
    "gate_fee": 50.0,
    "documentation_fee": 100.0,
    "per_mile_truck_rate": 2.0,
    "default_truck_distance_miles": 100.0,
    "ocean_freight_roro": 1200.0,
    "ocean_freight_container": 2200.0,
    "port_charges": 250.0,
    "marine_insurance_pct": 0.01,
    "clearing_percent_of_declared": 0.10,
    "vat_pct": 0.075
}

# -------------------------
# Utility / core functions
# -------------------------
def to_ngn(amount_foreign: float, fx_rate: float) -> float:
    return amount_foreign * fx_rate

def try_fetch_copart_lot(url_or_lot: str) -> Optional[LotData]:
    """
    Attempt to fetch and parse Copart lot page.
    This is a best-effort implementation: many sites block scraping. The function is opt-in
    and returns None if fetch/parsing fails. We avoid aggressive scraping; prefer using public APIs or user-supplied fields.
    """
    try:
        # If user provided only a lot number, try to construct a URL pattern (simple heuristic)
        if url_or_lot.isdigit():
            url = f"https://www.copart.com/lot/{url_or_lot}"
        else:
            url = url_or_lot
        headers = {"User-Agent": "DOL-Bids-App/1.0 (+https://example.com)"}
        resp = requests.get(url, headers=headers, timeout=8)
        if resp.status_code != 200:
            logger.info("Fetch failed or blocked (status %s). Falling back to manual input.", resp.status_code)
            return None
        text = resp.text
        # Heuristic parsing: look for JSON snippets or meta tags. Real parser should be robust and respect ToS.
        # For MVP, we try a few simple patterns; otherwise return None to prompt manual entry.
        # NOTE: This is intentionally conservative.
        if "vehicle" in text.lower() or "lot" in text.lower():
            # create a modest mocked LotData from limited parsing
            ld = LotData(url=url, lot_id=url.split("/")[-1][:12], make="(parsed?)", model="(parsed?)", year=None, currency=DEFAULT_CONFIG["currency"])
            logger.info("Basic parsing succeeded; returning minimal LotData (fields editable).")
            return ld
        return None
    except Exception as e:
        logger.debug("Exception during fetch: %s", e)
        return None

def estimate_repairs_from_damage(primary_damage: str, secondary_damage: Optional[str]) -> RepairEstimate:
    """
    Heuristic repair estimator. Produces low/base/high and line items in lot currency.
    Editable in UI.
    """
    items = []
    p = (primary_damage or "").lower()
    s = (secondary_damage or "").lower()
    # heuristics
    if "front" in p or "bumper" in p:
        items.append(RepairLineItem("Front bumper assembly", 1, 400.0, 3.0, 60.0))
        items.append(RepairLineItem("Headlight", 1, 150.0, 1.5, 60.0))
    if "fender" in p or "left" in p:
        items.append(RepairLineItem("Left fender", 1, 200.0, 2.0, 60.0))
    if not items:
        items.append(RepairLineItem("Minor bodywork & paint", 1, 600.0, 6.0, 60.0))
    base = sum(i.total_cost() for i in items)
    low = base * 0.7
    high = base * 1.5
    return RepairEstimate(low=low, base=base, high=high, items=items, contingency_pct=0.15)

def calc_copart_fees(hammer: float, fee_table=None) -> Dict[str, float]:
    if fee_table is None:
        fee_table = DEFAULT_CONFIG["copart_fee_table"]
    fee = 0.0
    for (mn, mx, f) in fee_table:
        if mn <= hammer <= mx:
            fee = f
            break
    internet = DEFAULT_CONFIG["internet_bid_fee"]
    gate = DEFAULT_CONFIG["gate_fee"]
    doc = DEFAULT_CONFIG["documentation_fee"]
    total = fee + internet + gate + doc
    return {"copart_tier_fee": fee, "internet_fee": internet, "gate_fee": gate, "documentation_fee": doc, "total_copart_fees": total}

def calc_trucking(distance_miles: Optional[float] = None) -> float:
    if distance_miles is None:
        distance_miles = DEFAULT_CONFIG["default_truck_distance_miles"]
    rate = DEFAULT_CONFIG["per_mile_truck_rate"]
    return distance_miles * rate

def calc_shipping_and_clearing(hammer: float, method: str = "RORO", declared_value: Optional[float] = None) -> Dict[str, float]:
    freight = DEFAULT_CONFIG["ocean_freight_roro"] if method.upper() == "RORO" else DEFAULT_CONFIG["ocean_freight_container"]
    port = DEFAULT_CONFIG["port_charges"]
    cif = (declared_value if declared_value is not None else hammer) + freight + port
    insurance = cif * DEFAULT_CONFIG["marine_insurance_pct"]
    clearing = (declared_value if declared_value is not None else hammer) * DEFAULT_CONFIG["clearing_percent_of_declared"]
    vat = (declared_value if declared_value is not None else hammer) * DEFAULT_CONFIG["vat_pct"]
    total = freight + port + insurance + clearing + vat
    return {"freight": freight, "port_charges": port, "insurance": insurance, "clearing_estimate": clearing, "vat_estimate": vat, "total_shipping_and_clearing": total, "CIF": cif}

def back_solve_hammer_max(resale_ngn: float, target_profit_ngn: float, fx_rate: float,
                          cad_costs_exhammer: float, repair_cad: float, ngn_native_costs: float, reserve_pct: float) -> float:
    """
    Hammer_CAD_max = (Resale_NGN - TargetProfit_NGN - NGN_native_costs - Reserve_NGN)/FX - (CAD_Costs_ExHammer + Repair_CAD)
    Reserve_NGN = (Repair_CAD * FX) * reserve_pct
    """
    reserve_ngn = repair_cad * fx_rate * reserve_pct
    numerator = resale_ngn - target_profit_ngn - ngn_native_costs - reserve_ngn
    hammer_max = numerator / fx_rate - (cad_costs_exhammer + repair_cad)
    return hammer_max

def profit_math(resale_ngn: float, fx_rate: float, hammer: float, cad_costs_exhammer: float,
                repair_cad: float, ngn_native_costs: float, reserve_pct: float) -> Dict[str, Any]:
    repair_ngn = repair_cad * fx_rate
    reserve_ngn = repair_ngn * reserve_pct
    all_in_ngn = (hammer + cad_costs_exhammer + repair_cad) * fx_rate + ngn_native_costs + reserve_ngn
    expected_profit = resale_ngn - all_in_ngn
    return {"all_in_ngn": all_in_ngn, "expected_profit_ngn": expected_profit, "repair_ngn": repair_ngn, "reserve_ngn": reserve_ngn}

# -------------------------
# Streamlit UI
# -------------------------
st.set_page_config(page_title="DOL Bids", layout="wide", initial_sidebar_state="expanded")
st.title("DOL Bids — Valuation & Bid Assistant")

# Sidebar: editable global defaults
st.sidebar.header("Global settings & defaults")
currency = st.sidebar.selectbox("Lot currency", options=["CAD", "USD"], index=0)
fx_rate = st.sidebar.number_input("Black-market FX (NGN per lot currency)", value=float(DEFAULT_CONFIG["fx_rate_ngn_per_lot"]), step=1.0, format="%.2f")
fx_ts = DEFAULT_CONFIG["fx_timestamp"]
target_profit_min = st.sidebar.number_input("Target profit min (NGN)", value=float(DEFAULT_CONFIG["target_profit_min_ngn"]), step=100000.0, format="%.0f")
target_profit_max = st.sidebar.number_input("Target profit max (NGN)", value=float(DEFAULT_CONFIG["target_profit_max_ngn"]), step=100000.0, format="%.0f")
reserve_pct = st.sidebar.slider("Reserve % of repair (for contingencies)", min_value=0.0, max_value=0.5, value=DEFAULT_CONFIG["reserve_pct"], step=0.01)
st.sidebar.markdown("**Shipping defaults**")
truck_rate = st.sidebar.number_input("Trucking rate (lot currency / mile)", value=float(DEFAULT_CONFIG["per_mile_truck_rate"]), step=0.1)
DEFAULT_CONFIG["per_mile_truck_rate"] = truck_rate
DEFAULT_CONFIG["currency"] = currency

# Main: input lot
st.header("1) Supply a Copart URL or Lot Number")
with st.form("lot_input_form"):
    col1, col2 = st.columns([3, 2])
    with col1:
        lot_input = st.text_input("Copart URL or Lot number (paste URL or numeric lot id)", value="")
    with col2:
        fetch_opt = st.checkbox("Try fetch & parse (optional)", value=False)
    parse_btn = st.form_submit_button("Load lot (or open manual entry)")

lot = None
if parse_btn and lot_input:
    lot = try_fetch_copart_lot(lot_input) if fetch_opt else None
    if lot is None:
        st.info("Could not auto-parse the lot. Please fill fields manually in the manual-entry panel below.")
        # Create a basic manual entry form
        st.header("Manual Lot Entry (fill missing fields)")
        with st.form("manual_lot_form"):
            year = st.number_input("Year", value=2016, min_value=1980, max_value=2030)
            make = st.text_input("Make", value="Toyota")
            model = st.text_input("Model", value="Camry")
            mileage = st.number_input("Mileage (km)", value=100000)
            primary_damage = st.text_input("Primary damage (short)", value="Front End")
            title = st.selectbox("Title type", options=["Salvage", "Clean", "Rebuilt", "Insurance"], index=0)
            run_and_drive = st.checkbox("Run & drive", value=False)
            submit_manual = st.form_submit_button("Save manual lot")
        if submit_manual:
            lot = LotData(url=lot_input or None, lot_id=lot_input, year=year, make=make, model=model,
                          mileage=mileage, primary_damage=primary_damage, title=title, run_and_drive=run_and_drive, currency=currency)
            st.success("Manual lot saved. Continue to assumptions & analysis.")
    else:
        st.success("Lot parsed (minimal fields). Please review and edit below.")

# If fetch succeeded we still allow editing
if lot:
    st.subheader("Lot summary (editable)")
    # present editable widget fields for the lot
    col1, col2, col3 = st.columns(3)
    with col1:
        lot.year = st.number_input("Year", value=lot.year or 2016, min_value=1980, max_value=2030)
        lot.make = st.text_input("Make", value=lot.make or "")
        lot.model = st.text_input("Model", value=lot.model or "")
    with col2:
        lot.trim = st.text_input("Trim", value=lot.trim or "")
        lot.vin = st.text_input("VIN", value=lot.vin or "")
        lot.mileage = st.number_input("Mileage (km)", value=int(lot.mileage or 0))
    with col3:
        lot.primary_damage = st.text_input("Primary damage", value=lot.primary_damage or "")
        lot.secondary_damage = st.text_input("Secondary damage", value=lot.secondary_damage or "")
        lot.title = st.text_input("Title", value=lot.title or "")
    st.markdown("---")

    # Assumptions & overrides panel
    st.header("2) Assumptions & Overrides")
    col1, col2, col3 = st.columns([2, 2, 2])
    with col1:
        resale_inputs = st.text_input("Resale scenarios (comma-separated ₦ amounts)", value="{},{}".format(*DEFAULT_CONFIG["resale_scenarios_ngn"]))
        resale_scenarios = [int(r.strip()) for r in resale_inputs.split(",") if r.strip()]
        shipping_method = st.selectbox("Shipping method", options=["RORO", "CONTAINER"], index=0)
        trucking_distance = st.number_input("Trucking distance (miles)", value=int(DEFAULT_CONFIG["default_truck_distance_miles"]))
    with col2:
        copart_fee_table = DEFAULT_CONFIG["copart_fee_table"]  # editable advanced could be implemented
        internet_fee = st.number_input("Internet bid fee (lot currency)", value=float(DEFAULT_CONFIG["internet_bid_fee"]), step=1.0)
        gate_fee = st.number_input("Gate fee (lot currency)", value=float(DEFAULT_CONFIG["gate_fee"]), step=1.0)
        doc_fee = st.number_input("Documentation fee (lot currency)", value=float(DEFAULT_CONFIG["documentation_fee"]), step=1.0)
    with col3:
        reserve_pct = st.slider("Reserve %", min_value=0.0, max_value=0.5, value=float(reserve_pct), step=0.01)
        fx_rate = st.number_input("FX (NGN per lot currency)", value=float(fx_rate), step=1.0, format="%.2f")
        st.markdown("FX timestamp: " + fx_ts)

    # Repair estimate & overrides
    st.header("3) Repair estimate (auto + manual overrides)")
    repairs = estimate_repairs_from_damage(lot.primary_damage or "", lot.secondary_damage or "")
    st.write("Automatic repair estimate (editable):")
    # show items in an editable table
    repair_df = pd.DataFrame([{
        "part": it.part,
        "qty": it.qty,
        "unit_cost": it.unit_cost,
        "labor_hours": it.labor_hours,
        "labor_rate": it.labor_rate,
        "total_cost": it.total_cost()
    } for it in repairs.items])
    edited_repairs = st.data_editor(repair_df, num_rows="dynamic")
    # allow user to edit numeric totals and recalc
    if edited_repairs is not None and len(edited_repairs) > 0:
        items_updated = []
        for _, r in edited_repairs.iterrows():
            items_updated.append(RepairLineItem(part=str(r["part"]), qty=int(r["qty"]),
                                                unit_cost=float(r["unit_cost"]), labor_hours=float(r["labor_hours"]),
                                                labor_rate=float(r["labor_rate"])))
        repairs.items = items_updated
        repairs.base = sum(i.total_cost() for i in repairs.items)
        repairs.low = repairs.base * 0.7
        repairs.high = repairs.base * 1.5

    st.markdown(f"- Low: {repairs.low:.2f} {currency}  |  Base: {repairs.base:.2f} {currency}  |  High: {repairs.high:.2f} {currency}")
    st.markdown("---")

    # Run analysis button
    st.header("4) Analyze & Suggest Bids")
    if st.button("Compute Avg & Max Bids"):
        # Build cost building blocks
        # Copart fees placeholder (we will compute using back-solve hammer; for cad_costs_exhammer we use shipping/trucking/fees placeholders)
        truck_cost = calc_trucking(trucking_distance)
        shipping_breakdown = calc_shipping_and_clearing(hammer=0, method=shipping_method)  # uses placeholders
        cad_costs_exhammer = (internet_fee + gate_fee + doc_fee) + truck_cost + shipping_breakdown["freight"] + shipping_breakdown["port_charges"] + shipping_breakdown["insurance"]
        ngn_native_costs = (shipping_breakdown["clearing_estimate"] + shipping_breakdown["vat_estimate"]) * fx_rate
        # Prepare results container
        scenario_cards = []
        for resale in resale_scenarios:
            # optimistic (base repair, current fx, lower target profit)
            hammer_max_opt = back_solve_hammer_max(
                resale_ngn=resale,
                target_profit_ngn=float(target_profit_min),
                fx_rate=fx_rate,
                cad_costs_exhammer=cad_costs_exhammer,
                repair_cad=repairs.base,
                ngn_native_costs=ngn_native_costs,
                reserve_pct=reserve_pct
            )
            # conservative (high repair, worse fx +5%, higher target profit)
            fx_worse = fx_rate * 1.05
            hammer_max_cons = back_solve_hammer_max(
                resale_ngn=resale,
                target_profit_ngn=float(target_profit_max),
                fx_rate=fx_worse,
                cad_costs_exhammer=cad_costs_exhammer,
                repair_cad=repairs.high,
                ngn_native_costs=ngn_native_costs,
                reserve_pct=reserve_pct
            )
            avg_bid = (hammer_max_opt + hammer_max_cons) / 2.0
            max_bid = max(0.0, hammer_max_opt, hammer_max_cons)

            # compute fee breakdowns for display (using the computed hammer values)
            fees_opt = calc_copart_fees(max(0.0, hammer_max_opt))
            shipping_opt = calc_shipping_and_clearing(max(0.0, hammer_max_opt), method=shipping_method)
            profit_opt = profit_math(resale, fx_rate, max(0.0, hammer_max_opt), cad_costs_exhammer, repairs.base, ngn_native_costs, reserve_pct)

            fees_cons = calc_copart_fees(max(0.0, hammer_max_cons))
            shipping_cons = calc_shipping_and_clearing(max(0.0, hammer_max_cons), method=shipping_method)
            profit_cons = profit_math(resale, fx_worse, max(0.0, hammer_max_cons), cad_costs_exhammer, repairs.high, ngn_native_costs, reserve_pct)

            scenario_cards.append({
                "resale_ngn": resale,
                "avg_bid": avg_bid,
                "max_bid": max_bid,
                "hammer_opt": hammer_max_opt,
                "profit_opt": profit_opt,
                "fees_opt": fees_opt,
                "shipping_opt": shipping_opt,
                "profit_cons": profit_cons,
                "fees_cons": fees_cons,
                "shipping_cons": shipping_cons,
                "cad_costs_exhammer": cad_costs_exhammer,
                "ngn_native_costs": ngn_native_costs,
                "repairs": repairs
            })

        # Display results top-line
        st.subheader("Suggested Bids (top-line)")
        cols = st.columns(len(scenario_cards))
        for i, sc in enumerate(scenario_cards):
            with cols[i]:
                st.metric(label=f"Resale ₦{sc['resale_ngn']:,}", value=f"Avg bid: {sc['avg_bid']:.2f} {currency}", delta=f"Max {sc['max_bid']:.2f} {currency}")

        st.markdown("---")
        st.subheader("Detailed scenario breakdowns")

        # Show detailed expansion for each scenario
        for sc in scenario_cards:
            st.markdown(f"### Scenario — Resale ₦{sc['resale_ngn']:,}")
            left, right = st.columns([2,1])
            with left:
                st.write("**Optimistic case (base repair, current FX)**")
                st.write(f"Suggested hammer (CAD): {sc['hammer_opt']:.2f}")
                st.write("Expected profit (NGN): {:.2f}".format(sc['profit_opt']['expected_profit_ngn']))
                fee_df_opt = pd.DataFrame([sc['fees_opt']])
                st.dataframe(fee_df_opt.T.rename(columns={0:"value"}))
                ship_df_opt = pd.DataFrame([sc['shipping_opt']])
                st.dataframe(ship_df_opt.T.rename(columns={0:"value"}))
                st.write(f"Repairs (base): {sc['repairs'].base:.2f} {currency}")
            with right:
                st.write("**Conservative case (high repair, FX+5%)**")
                st.write(f"Suggested hammer (CAD): {sc['max_bid']:.2f}")
                st.write("Expected profit (NGN): {:.2f}".format(sc['profit_cons']['expected_profit_ngn']))
                fee_df_cons = pd.DataFrame([sc['fees_cons']])
                st.dataframe(fee_df_cons.T.rename(columns={0:"value"}))
                ship_df_cons = pd.DataFrame([sc['shipping_cons']])
                st.dataframe(ship_df_cons.T.rename(columns={0:"value"}))
                st.write(f"Repairs (high): {sc['repairs'].high:.2f} {currency}")

            # Export buttons
            csv_buf = io.StringIO()
            export_rows = {
                "resale_ngn": sc["resale_ngn"],
                "avg_bid": sc["avg_bid"],
                "max_bid": sc["max_bid"],
                "hammer_opt": sc["hammer_opt"],
                "expected_profit_opt_ngn": sc["profit_opt"]["expected_profit_ngn"],
                "repairs_base_cad": sc["repairs"].base,
                "cad_costs_exhammer": sc["cad_costs_exhammer"],
            }
            df_exp = pd.DataFrame([export_rows])
            df_exp.to_csv(csv_buf, index=False)
            st.download_button(label="Download scenario CSV", data=csv_buf.getvalue(), file_name=f"scenario_{sc['resale_ngn']}_export.csv", mime="text/csv")

        # Sensitivity Visualization
        st.markdown("---")
        st.subheader("Sensitivity: FX swings vs Max Bid")
        fx_vals = [fx_rate * (1 - 0.10), fx_rate, fx_rate * (1 + 0.10)]
        sens_rows = []
        for sc in scenario_cards:
            for fx in fx_vals:
                hammer_max = back_solve_hammer_max(resale_ngn=sc["resale_ngn"], target_profit_ngn=float(target_profit_min),
                                                  fx_rate=fx, cad_costs_exhammer=sc["cad_costs_exhammer"],
                                                  repair_cad=sc["repairs"].base, ngn_native_costs=sc["ngn_native_costs"],
                                                  reserve_pct=reserve_pct)
                sens_rows.append({"resale": sc["resale_ngn"], "fx": fx, "hammer_max": hammer_max})
        sens_df = pd.DataFrame(sens_rows)
        # Plot with matplotlib (no explicit colors)
        fig, ax = plt.subplots()
        for resale_val, group in sens_df.groupby("resale"):
            ax.plot(group["fx"], group["hammer_max"], marker="o", label=f"Resale ₦{resale_val:,}")
        ax.set_xlabel("FX (NGN / lot currency)")
        ax.set_ylabel(f"Max Hammer ({currency})")
        ax.set_title("Sensitivity: FX vs Max Bid")
        ax.legend()
        st.pyplot(fig)

        st.success("Analysis complete. Use export buttons above to download CSVs or copy values.")
        st.info("If web fetch failed earlier, consider filling more precise values (declared value, port, trucking distance, BDC FX) for better accuracy.")

    # End of compute block
else:
    st.info("Start by entering a Copart URL or lot number and press 'Load lot' or fill manual entry.")

# -------------------------
# Footer: deployment & next steps
# -------------------------
st.markdown("---")
st.header("Deployment & Packaging (next steps)")
st.markdown("""
**Local run:**  
- `pip install streamlit pandas matplotlib requests`  
- `streamlit run app.py`

**Deploy to the web (recommended for app-store wrappers):**  
- Option A: Streamlit Cloud (fast): push repo to GitHub and deploy via Streamlit Cloud.  
- Option B: Docker on VPS (more control): create Dockerfile, run on DigitalOcean / AWS / GCP, front via domain + TLS.

**Make available on App Stores:**  
- Wrap your hosted web app inside a WebView/TWA:
  - Android: Trusted Web Activity (Chrome TWA) or simple WebView wrapper; publish APK/AAB to Google Play. Tools: `bubblewrap`, `pwabuilder`.  
  - iOS: Wrap in a simple WKWebView native container (Xcode) and publish via App Store / TestFlight (note Apple policy may prefer native features).  
- Alternatively build a React Native / Capacitor / Ionic native shell pointing to your hosted URL for more native control.

**Security & production notes:**  
- Protect API keys and Copilot keys in `.env` (do not hardcode).  
- Use HTTPS and authentication if you expose this publicly.  
- Respect Copart ToS — if scraping, obtain permissions; otherwise prefer asking users to paste lot details or use official APIs.
""")



ModuleNotFoundError: No module named 'streamlit'

In [6]:
import sys
print(sys.executable)


c:\Users\OLADEINDE\Downloads\DOLBids\.venv\Scripts\python.exe


In [7]:
!{sys.executable} -m pip show streamlit


In [8]:
!{sys.executable} -m pip install streamlit


  Using cached streamlit-1.49.1-py3-none-any.whl.metadata (9.5 kB)
  Using cached altair-5.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached cachetools-6.2.0-py3-none-any.whl.metadata (5.4 kB)
  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached numpy-2.3.2-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached pandas-2.3.2-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached pillow-11.3.0-cp313-cp313-win_amd64.whl.metadata (9.2 kB)
  Using cached protobuf-6.32.0-cp310-abi3-win_amd64.whl.metadata (593 bytes)
  Using cached pyarrow-21.0.0-cp313-cp313-win_amd64.whl.metadata (3.4 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached watchdog-6.0.0-py3-none-wi

In [9]:
!{sys.executable} -m streamlit --version


Streamlit, version 1.49.1


In [10]:
!{sys.executable} -m streamlit run app.py --server.port 8501


Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: app.py


In [11]:
import os
print(os.getcwd())


c:\Users\OLADEINDE\Downloads\DOLBids\Dimex Bids\app.py


In [12]:
!{sys.executable} -m streamlit run "c:\Users\OLADEINDE\Downloads\DOLBids\Dimex Bids\app.py" --server.port 8501


^C


In [ ]:
pip install streamlit pandas matplotlib requests


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Windows
venv\Scripts\activate

# macOS/Linux
source venv/bin/activate


SyntaxError: unexpected character after line continuation character (3636110513.py, line 2)